## Notebook to classify snow-covered area (SCA) and determine minimum snow elevations in PlanetScope 4-band images

Rainey Aberle, 2022

### Inputs required:

- __Area of interest (AOI) (`.shp` file):__ can be used to crop images before classifying snow
- __Google Earth Engine account:__ sign up for a free account [here](https://earthengine.google.com/new_signup/). Used for accessing global digital elevation model datasets.  
- __PlanetScope images (`.tif` file(s)):__ used to classify snow
- __Trained classifier (`.sav` file):__ used to classify snow in PlanetScope images (find in `planet-snow/inputs-outputs/` or develop in `develop_classifier.ipynb`)

### 1. Import packages

In [ ]:
import os
import ee
import glob
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio as rio
import fiona
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt, dates
from shapely.geometry import shape, Polygon
from scipy import stats
import pickle
import sys
import time

### 2. Define settings and paths in directory

In [ ]:
# -----Define site name (used for folder and output file names)
# Wolverine, Mendenhall, SitKusa, Easton, SCascade_LeConte, Blue, Emmons
site_name = 'SCascade_LeConte'

# -----Define settings
save_outputs = True # = True to save SCA images to file
save_figures = True # = True to save output figures
crop_to_AOI = False # = True to crop images to AOI before calculating SCA

# -----Define paths in directory
# base directory (path to "planet-snow/")
base_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/planet-snow/'
# image directory
im_path = base_path+'../study-sites/'+site_name+'/imagery/Planet/adjusted-filtered/'
# SCA images output directory 
out_path = base_path+'../study-sites/'+site_name+'/imagery/Planet/classified/'
# figures output directory (only required if save_figures==True)
figures_out_path = base_path+'../study-sites/'+site_name+'/figures/'
# AOI shapefile full path in directory
# AOI_fn = base_path+'../../GIS_data/RGI_outlines/'+site_name+'_RGI.shp'
AOI_fn = base_path+'../../GIS_data/SCascade_LeConte_bounding_box.shp'

### 4. Load files

- Area of interest (AOI)
- PlanetScope images
- Image classifier

In [ ]:
# -----Load image names from file
os.chdir(im_path) # change directory
im_fns = glob.glob('*.tif') # file names for all .tif files in im_path
im_fns.sort() # sort file names by date

# -----Load image classifier and feature columns
clf_fn = base_path+'inputs-outputs/'+'all_sites_classifier.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/all_sites_classifier_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Load the AOI as gpd.DataFrame
AOI = gpd.read_file(AOI_fn)

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from classification_utils import query_GEE_for_DEM, crop_images_to_AOI, classify_image, calculate_SCA, determine_snow_elevs 

### 4. Authenticate and initialize GEE, load the DEM

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back in this notebook when prompted. 

In [ ]:
# -----Authenticate and initialize
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()
    
# -----Load the DEM
DEM, DEM_x, DEM_y, AOI_UTM = query_GEE_for_DEM(AOI, im_path, im_fns)

### 5. Classify snow, calculate SCA, determine minimum snow elevation in each image

In [ ]:
# -----Start timer
t1 = time.time()

# -----Initialize image dates
im_dts = [] # image datetimes

# -----Crop images if previously selected
if crop_to_AOI==True:
    
    # Crop images if previously selected
    cropped_im_path = crop_images_to_AOI(im_path, im_fns, AOI_UTM)
    # grab cropped image names
    os.chdir(cropped_im_path) # change directory
    im_fns_crop = glob.glob('*_crop.tif')
    im_fns_crop.sort() # sort file names by date
    im_fns_loop = im_fns_crop # image file names to use in loop
    
else:
    
    os.chdir(im_path)
    im_fns_loop = im_fns # im_names to use in loop

# -----Make directory for output figures (if it does not already exist in file)
if save_figures and os.path.exists(figures_out_path)==False:
    os.mkdir(figures_out_path)
    print('made directory for output figures:' + figures_out_path)
        
# -----Create figure for snow elevations box plot
fig2, ax = plt.subplots(figsize=(16,8))
ax.set_ylabel('Snow elevations [m a.s.l.]')
ax.xaxis.set_major_formatter(dates.DateFormatter('%Y'))

# -----Initialize DataFrame to hold stats summary
df = pd.DataFrame(columns=('site_name', 'datetime', 'im_elev_min', 'im_elev_max', 'snow_elev_min', 'snow_elev_max', 
                           'snow_elev_median', 'snow_elev_10th_perc', 'snow_elev_90th_perc'))

# -----Loop through images
i=0 # loop counter
for im_fn in im_fns_loop:

    # extract datetime from image name
    im_dt = np.datetime64(im_fn[0:4] + '-' + im_fn[4:6] + '-' + im_fn[6:8]
                          + 'T' + im_fn[9:11] + ':00:00')
    im_dts = im_dts + [im_dt]

    # open image
    im = rio.open(im_fn)

    # classify snow
    im_x, im_y, im_classified = classify_image(im, im_fn, clf, feature_cols, out_path)   
    
    # determine snow elevations
    plot_output = True
    im_elev_min, im_elev_max, snow_elev, fig = determine_snow_elevs(DEM, DEM_x, DEM_y, im, im_classified, im_dt, im_x, im_y, plot_output)
#     plot_output = False
#     snow_elev = determine_snow_elevs(DEM, DEM_x, DEM_y, snow, im, im_dt, im_x, im_y, plot_output)
    
    # calculate and plot stats
    iqr = stats.iqr(snow_elev, 
                rng=(10, 90))
    med = np.median(snow_elev)
    ax.add_patch(Rectangle((im_dt-np.timedelta64(1, 'D'), med-iqr/2), 
                           width=2*np.timedelta64(1, 'D'), height=iqr, color='blue'))
    ax.scatter([im_dt, im_dt], [np.min(snow_elev), np.max(snow_elev)], color='blue', s=10)
    ax.scatter(im_dt, med, facecolor='white', edgecolor='black', s=20)

    # save stats in pandas DataFrame
    df_row = pd.DataFrame({'site_name':site_name, 'datetime':im_dt, 'im_elev_min':im_elev_min, 'im_elev_min':im_elev_max, 
                           'snow_elev_min':np.min(snow_elev), 'snow_elev_min':np.max(snow_elev), 'snow_elev_median':med,  
                           'snow_elev_10th_perc':med-iqr/2, 'snow_elev_90th_perc':med+iqr/2}, index=[0])
    df = pd.concat([df, df_row], ignore_index=True)
    
    # save figure
    if save_figures==True:
        fig.savefig(figures_out_path+im_fn[0:15]+'_SCA.png', dpi=200, facecolor='white', edgecolor='none')
        print('figure saved to file')

    i+=1 # increase loop counter

# -----Save figure and data table
if plot_output and save_figures:
    fig2.savefig(figures_out_path+site_name+'_snow_elevs.png', dpi=200, facecolor='white', edgecolor='none')
    print('snow elevations figure saved to file')
if save_outputs:
    df.to_csv(path_or_buf=out_path+site_name+'_snow_elevs_stats.csv', sep=',', na_rep='', header=True)
    print('data table saved to file')

# -----Stop timer
print('Time elapsed: '+str(np.round((time.time()-t1)/60, 2))+' minutes')

# -----Display complete figure 2
fig2